In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

data = pd.read_excel (r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=0)

m = Model()

# data sets

supplycountry_names = data['SupplyCountry'].tolist() # names of energy supply countries: ["Angola","Algeria", ..]
supplycountry_indices = range(len(supplycountry_names)) # number of supply countries: [0,1,..]
demandcountry_names = data['DemandCountry'].tolist() # names of energy demand countries: ["Angola","Algeria", ..]
demandcountry_indices = range(len(demandcountry_names)) # number of demand countries: [0,1,..]
technology_names = data['Technologies'].tolist() # names of Technologies: ["Solar","Wind"]
technology_indices = range(len(technology_names)) # number of Technologies: ["Solar","Wind"]


# parameters:

supplysheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=1)
gencap = [] # generation capacity of supply country i for technology t
for i in supplycountry_names:
    gencap.append(supplysheet[(supplysheet['SupplyCountry'] == i)]['Supply'].tolist()) 
    
demandsheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=2)
d = demandsheet['Demand'].tolist()  # demand of demand country j (MW)

gencostsheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=3) # generation cost in supply country i using technology t
gencost = [] 
for i in supplycountry_names:
    gencost.append(gencostsheet[(gencostsheet['SupplyCountry'] == i)]['genCost'].tolist())
                   
# generation cost of each unit (e.g. MW) energy carrier in supply country i (million $/MW)

tcsheet = pd.read_excel (r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=4)  # transport cost from supply country i to demand country j (million $)
tc = []
for i in supplycountry_names:
    tc.append(tcsheet[(tcsheet['SupplyCountry'] == i)]['transCost'].tolist())

# decision variables

x_elec = m.addVars(supplycountry_indices, demandcountry_indices, technology_indices, name="transport")   # xijt = (quantity) amount of electricty transported from supply country i to demand country j with technology t (MW)


m.update()

# objective function
m.setObjective(sum(gencost[i][t]*x_elec[i,j,t] for i in supplycountry_indices for t in technology_indices for j in demandcountry_indices) + sum(tc[i][j]*x_elec[i,j,t] for i in supplycountry_indices for j in demandcountry_indices for t in technology_indices), GRB.MINIMIZE) # economic objective


# constraints

# 1.amount of electricity transferred to each demand country should be smaller or equal to the generation capacity of the supply country
m.addConstrs(
    sum(x_elec[i,j,t] for j in demandcountry_indices) <= gencap[i][t] 
    for i in supplycountry_indices for t in technology_indices
)
# 2.demand should be fulfilled by the electricity transferred from each supply country
m.addConstrs(
    sum(x_elec[i,j,t] for i in supplycountry_indices for t in technology_indices) == d[j]
    for j in demandcountry_indices
)


#3.each country should be able to satisfy 50% of its own demand

for i in supplycountry_indices:
    for j in demandcountry_indices:
        if (i == j):
            m.addConstr(sum(x_elec[i,j,t] for t in technology_indices) >= 0.5*d[j])

# optimization of model
m.optimize()

if m.status == GRB.OPTIMAL:
    print(f"Optimal solutions of transport cost: {m.ObjVal} ")
    
    m.getVars()

    for i in supplycountry_indices:
        summation = 0
        for j in demandcountry_indices:
            for t in technology_indices:
                summation += x_elec[i,j,t].x 
        if (summation> 0):
            print(f"Supply Country {i} provides electricity")

    print("Transport amount from supply country i to demand country j: ")
    for i in supplycountry_indices:
        for j in demandcountry_indices:
            for t in technology_indices:
                if (x_elec[i,j,t].x > 0):
                    print(f"from supply country {i} to demandcountry {j} with technology {t}: {x_elec[i,j,t].x}")
    for i in supplycountry_indices:
        total = 0
        for t in technology_indices:
            print (f"generation capacity of country {i} for technology {t} is: {gencap[i][t]}")
            total += gencap[i][t]
        print (f"total generation capacity of country {i} is: {total}")
else:
    print(m.status)

m.printAttr("X")
                    
m.write("simplemodel.lp")




Academic license - for non-commercial use only - expires 2021-04-02
Using license file C:\Users\karbassi\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 27 columns and 63 nonzeros
Model fingerprint: 0x117c4c4f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 9 rows and 19 columns
Presolve time: 0.01s
Presolved: 6 rows, 8 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9500000e+03   2.062500e+01   0.000000e+00      0s
       2    3.9500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective  3.950000000e+03
Optimal solutions of transport cost: 3950.0 
Supply Country 0 provides electricity
Supply Country 1 provides electricity
Supply Countr

In [ ]:
tcsheet[(tcsheet['SupplyCountry'] == 'Angola')]

In [ ]:
print (tcsheet)